In [1]:
import pandas as pd
import seaborn as sns

df = pd.read_csv('dulieuxettuyendaihoc.csv',header=0, delimiter=',',encoding='utf-8')

In [5]:
df.rename(columns={'TOANLOGICPHANTICH':'LOGIC',
                   'GIAIQUYETVANDE':'UNGXU',
                   'DINHHUONGNGHENGHIEP' : 'HUONGNGHIEP'},inplace=True)

In [3]:
#dữ liệu thì sinh thi khôi C
dfKhoiC = df.loc[df['KT'] == 'C']
dfKhoiC['LOGIC'].describe()

count    14.000000
mean      4.232143
std       1.947544
min       1.500000
25%       2.750000
50%       3.875000
75%       5.187500
max       8.000000
Name: LOGIC, dtype: float64

In [6]:
import scipy.stats as stats
# Với one sample T-test thì giả thiết Ho: muy = 4.0
# Mặc định mức tin cậy là 95% và mức sai lầm là 5%
stats.ttest_1samp(dfKhoiC['LOGIC'], popmean=4.0)

TtestResult(statistic=0.44599723713991907, pvalue=0.6629370899710998, df=13)

In [7]:
# TWO SAMPLE T-TEST
# Mục đích: Kiểm tra xem trung bình của 2 biến số (định lượng) có bằng nhau không
# Ho: delta_mean = 0
# Hãy kiểm tra xem trung bình điểm thi LOGIC và
# trung bình điểm thi UNGXU của thí sinh thi khối C có bằng nhau không?
# Ho: mean_LOGIC - mean_UNGXU = 0
dfKhoiC = df.loc[df['KT'] == 'C']
stats.ttest_ind(dfKhoiC['LOGIC'], dfKhoiC['UNGXU'], equal_var=True)

TtestResult(statistic=-1.0329196014245297, pvalue=0.3111543826061086, df=26.0)

In [15]:
from statsmodels.stats.weightstats import ztest as ztest
# Hãy kiểm tra xem trung bình điểm toán học kì 2 lớp 12 có bằng 8.0
ztest(df['T6'], value=8.0)
#bac bo

(-7.797828845339864, 6.298135014120743e-15)

In [16]:
# Hãy kiểm tra xem điểm trung bình toán học kì 1
# và học kì 2 năm lớp 12 có bằng nhau không
ztest(df['T5'], df['T6'], value=0)

(-1.094138573502891, 0.273894207026412)

In [ ]:
# Kiểm tra trung bình LOGIC và UNGXU có bằng nhau không
# với mức tin cậy 95%


In [17]:
from scipy.stats.stats import pearsonr
# Câu 5: Kiểm tra xem điểm toán học kì 1 và học kì 2 năm lớp 12 có tương quan không?
pearsonr(df['T5'], df['T6'])
#Dpearsonr = 0.77
# < 0.5 yeu
# 0.5 -> 0.65 TB
# 0.65 -> 0.75 Tot
#ket luan tren mau T5 T6 tuong quan thuan va muc do tuong quan rat tot.
#suy dien ra cho tong the ta thay ppl = 0.77 bac bo Aplay

C:\Users\Admin\AppData\Local\Temp\ipykernel_11996\4070253678.py:1: DeprecationWarning: Please use `pearsonr` from the `scipy.stats` namespace, the `scipy.stats.stats` namespace is deprecated.
  from scipy.stats.stats import pearsonr


PearsonRResult(statistic=0.7786831657869809, pvalue=1.4846407216273482e-21)

In [11]:
# Sinh viên làm tương tự cho T5 và LOGIC có tương quan hay không?
pearsonr(df['T5'], df['LOGIC'])
#khoong 

PearsonRResult(statistic=0.18464661226012727, pvalue=0.06590059130545516)

In [14]:
import scipy.stats as stats


# Hãy kiểm tra xem có sự phụ thuộc nào giữa việc sinh viên có định hướng nghề nghiệp
# và giới tính khi thí sinh đăng ký dự thi hay không
crosdata = pd.crosstab(df['GT'], [df['HUONGNGHIEP']], rownames=['GT'], colnames=['HUONGNGHIEP'])
crosdata



odd_ratio, p_value = stats.fisher_exact(crosdata)
print('odd ratio is : ' + str(odd_ratio))
print('p_value is : ' + str(p_value))

#chấp nhận 
# sai lầm 5%
# tin cậy 95%


odd ratio is : 0.575
p_value is : 0.22763927303454412


In [19]:
from scipy.stats import chi2_contingency
# Hãy kiểm tra xem có sự phụ thuộc nào giữa khối thi và khu vực khi đăng ký dự thi hay không
crosdata = pd.crosstab(df['KV'], [df['KT']], rownames=['KV'], colnames=['KT'])
crosdata

stat, p, dof, expected = chi2_contingency(crosdata)
 
# interpret p-value
alpha = 0.05
print("p value is " + str(p))
if p <= alpha:
    print('Dependent (reject H0)')
else:
    print('Independent (H0 holds true)')

#bác bỏ H0


p value is 0.02012461887796485
Dependent (reject H0)


In [20]:
# GT và KT có mối quan hệ hay không
crosdata = pd.crosstab(df['GT'], [df['KT']], rownames=['GT'], colnames=['KT'])
crosdata

stat, p, dof, expected = chi2_contingency(crosdata)
 
# interpret p-value
alpha = 0.05
print("p value is " + str(p))
if p <= alpha:
    print('Dependent (reject H0)')
else:
    print('Independent (H0 holds true)')
    
    # bác bỏ H0

p value is 0.005044752209452435
Dependent (reject H0)


In [21]:
# Điểm toán học kì 2 lớp 12 có phụ thuộc vào giới tính không
import statsmodels.api as sm
from statsmodels.formula.api import ols


model = ols('T6 ~ GT', data=df).fit()
aov_table = sm.stats.anova_lm(model, typ=1)
aov_table


,df,sum_sq,mean_sq,F,PR(>F)
GT,1.0,1.55201,1.55201,0.833769,0.363426
Residual,98.0,182.42109,1.86144,NaN,NaN


In [23]:
# Điểm LOGIC có phụ thuộc vào khu vực hay không
import statsmodels.api as sm
from statsmodels.formula.api import ols


model = ols('LOGIC ~ KV', data=df).fit()
aov_table = sm.stats.anova_lm(model, typ=1)
aov_table

# không 

,df,sum_sq,mean_sq,F,PR(>F)
KT,4.0,3.967636,0.991909,0.911814,0.46041
Residual,95.0,103.344864,1.087841,NaN,NaN


In [24]:
# Điểm UNGXU có phụ thuộc khối thi hay không
import statsmodels.api as sm
from statsmodels.formula.api import ols


model = ols('UNGXU ~ KT', data=df).fit()
aov_table = sm.stats.anova_lm(model, typ=1)
aov_table

#có

,df,sum_sq,mean_sq,F,PR(>F)
KT,4.0,3.967636,0.991909,0.911814,0.46041
Residual,95.0,103.344864,1.087841,NaN,NaN


In [25]:
# Hãy cho biết điểm LOGIC có phụ thuộc vào loại GT trên từng nhóm KV hay không
model = ols('LOGIC ~ GT + KV + GT:KV',data=df).fit()
result = sm.stats.anova_lm(model, type=2)
 
# Print the result
print(result)

#logic ->GT => p.value = 0.052 => Acept -> ko phụ thuộc
#Logic ->KV => p.value = 0.019 => Acept -> có phụ thuộc

            df     sum_sq   mean_sq         F    PR(>F)
GT         1.0   3.998401  3.998401  3.853364  0.052602
KV         2.0   8.561314  4.280657  4.125382  0.019173
GT:KV      2.0   1.149707  0.574854  0.554002  0.576510
Residual  94.0  97.538077  1.037639       NaN       NaN


In [26]:
#Phân tích xem NGONNGU có phụ thuộc trên nhóm KV theo từng loại KT hay khôn
model = ols('NGONNGU ~ KT + KV + KT:KV',data=df).fit()
result = sm.stats.anova_lm(model, type=2)
 
# Print the result
print(result)

#logic ->NGONNGU => p.value = 2.92 => Acept -> không phụ thuộc
#Logic ->KV => p.value = 2.7 => Acept -> có phụ thuộc

            df      sum_sq    mean_sq          F        PR(>F)
KT         4.0   65.783045  16.445761  10.991666  2.922883e-07
KV         2.0    3.885373   1.942686   1.298411  2.782013e-01
KT:KV      8.0    1.588377   0.198547   0.132701  9.976040e-01
Residual  87.0  130.169640   1.496203        NaN           NaN


In [ ]:
#PHÂN TÍCH HỒI QUY

#MỤC ĐÍCH : #phân tích ảnh hưởng (tác động) giữa các biến số định lượng (định lg)
#ví dụ : hãy phân tích sự tác dụng giữa điểm T5 T6 
# y = f(x) =. T6 = f(T5)
# T6 = phụ thuộc 
# T5 = độc lập giải thích
    

In [27]:
# Hãy cho biết sự ảnh hưởng của điểm học kì 1 năm lớp 12 đến điểm học kì 2 năm lớp 12
import statsmodels.api as sm


#adding a constant
X_with_constant = sm.add_constant(df[["T5"]].values)


y = df[['T6']].values


#performing the regression
result = sm.OLS(y, X_with_constant).fit()


# Result of statsmodels
print(result.summary())

#model : OLS
#R-squared : 0.6 cho 60% sự thay đổi của T6 có thể do T5
# 40% còn lại chưa biết
#adj R-squared
#Prod(F-stat) : 1.48e-21 mô hình này phù hợp
#const : 2.1130 A0
#x1 0.7182 A1
#T6 : 2.1130 + 0.7182 * T5
#nếu giá trị T6 mà được tính từ T5 theo phương trình trên ta gọi là giá trị dự báo



                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.606
Model:                            OLS   Adj. R-squared:                  0.602
Method:                 Least Squares   F-statistic:                     151.0
Date:                Sat, 09 Sep 2023   Prob (F-statistic):           1.48e-21
Time:                        08:23:15   Log-Likelihood:                -125.76
No. Observations:                 100   AIC:                             255.5
Df Residuals:                      98   BIC:                             260.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.1130      0.402      5.257      0.0

In [28]:
#T5 = 70 
#T6 = product = 7.14
# P > |t| < Aplay 

# Sinh viên tự khám phá sự ảnh hưởng của T6 đến điểm thi LOGIC
import statsmodels.api as sm


#adding a constant
X_with_constant = sm.add_constant(df[["LOGIC"]].values)


y = df[['T6']].values


#performing the regression
result = sm.OLS(y, X_with_constant).fit()


# Result of statsmodels
print(result.summary())

#R-squared : 0.09 cho 9% sự thay đổi của T6 có thể do LOGIC
# 81% còn lại chưa biết


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.091
Model:                            OLS   Adj. R-squared:                  0.082
Method:                 Least Squares   F-statistic:                     9.798
Date:                Sat, 09 Sep 2023   Prob (F-statistic):            0.00230
Time:                        08:37:47   Log-Likelihood:                -167.61
No. Observations:                 100   AIC:                             339.2
Df Residuals:                      98   BIC:                             344.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.2873      0.543      9.738      0.0

In [34]:
# Hãy phân tích sự ảnh hưởng của điểm toán học kì 1,2 năm lớp 12 đến điểm LOGIC
import statsmodels.api as sm


#adding a constant
X_with_constant = sm.add_constant(df[["LOGIC"]].values)


y = df.groupby('T6')['T5'].value_counts

#y = df[['T6','T5']].values


#performing the regression
result = sm.OLS(y, X_with_constant).fit()


# Result of statsmodels
print(result.summary())

# Cho biết nếu T5 = 6.0 và T6 = 5.5 thì dự đoán học sinh đi thi sẽ được điểm LOGIC là nhiêu


TypeError: object of type 'method' has no len()